In [1]:
from IPython.display import Markdown, display, Audio
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import Javascript

import sys
import glob, os

import papermill as pm

import mne
from mne.filter import filter_data
from mne.preprocessing import ICA
from mne import create_info, Annotations
from mne.io import RawArray

from mne_icalabel import label_components

from autoreject import AutoReject
from autoreject import Ransac  # noqa
from autoreject.utils import interpolate_bads  # noqa

import numpy as np
import torch
import pandas as pd
from matplotlib import pyplot as plt

# sys.path.insert(0, os.path.abspath('../../asrpy/'))
# from asrpy import ASR

sys.path.insert(0, os.path.abspath('../picard'))
import picard

C:\Program Files\Python310\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

In [3]:
# blocks parasitic prints from functions. Run enablePrint() to re-authorize it.
# blockPrint()
# enablePrint()

In [4]:
# np.set_printoptions(threshold=np.inf)

## Functions

In [5]:
def check_markers(events, original_marker_list):
    idx_new_segment = [i for i, x in enumerate(events) if x[2] == 99999]
    if len(idx_new_segment) > 1:
        events = events[count_new_segment[2]:]

    count_sound_in = len([i for i, x in enumerate(events) if x[2] == 2])
    count_frequent = len([i for i, x in enumerate(events) if x[2] == original_marker_list[0]])
    count_deviant = len([i for i, x in enumerate(events) if x[2] == original_marker_list[1]])
    
    counts = pd.DataFrame.from_dict({'new_segment' : [len(idx_new_segment)],
                                     'sound_in' : [count_sound_in],
                                     'frequent' : [count_frequent],
                                     'deviant' : [count_deviant]})
    return counts

In [6]:
def checkNan(EEG):
    # Check for NaN values in the raw data
    nan_indices = np.argwhere(np.isnan(EEG.get_data()))

    # Print the number of NaN values and their indices
    if nan_indices.size == 0:
        print("No NaN values found in the data.")
    else:
        print("Found {} NaN values in the data at the following indices:".format(len(nan_indices)))
        for i in range(len(nan_indices)):
            print("Sample {}: Channel {}".format(nan_indices[i][1], nan_indices[i][0]))
            
        EEG.crop(tmin=EEG.times[3400000], tmax=EEG.times[-1])
        EEG.crop(tmin=2.0, tmax=EEG.times[50200])
    
    sum_nan_2 = len(np.argwhere(np.isnan(EEG.get_data())))
    return EEG, sum_nan_2

In [7]:
def take_sound_in(events):
    events_df = pd.DataFrame(events, columns=['onset', 'duration', 'event_type'])
    event_type_dict = {99999:'new_segment', 1013: 'deviant', 1012: 'frequent', 2: 'sound_in', 1011:'block',
                      1015:0, 1001:1, 1002:2, 1003:3, 1004:4, 1005:5, 1006:6, 1007:7, 1008:8, 1009:9}
    events_df['event_name'] = np.vectorize(event_type_dict.get)(events_df['event_type'])

    # Initialize a new column called 'event_number' with empty strings
    events_df['event_number'] = ''

    # Loop through the rows of the dataframe
    for i, row in events_df.iterrows():
        # Check if the current row is a 'frequent' or 'deviant'
        if row['event_name'] in ['frequent', 'deviant']:
            # Get the next four rows
            next_rows = events_df.iloc[i+1:i+5]
            # Concatenate the 'event_name' column values of the next four rows
            event_numbers = ''.join([str(val) for val in next_rows['event_name'].tolist()])
            # Update the 'event_number' value of the current row
            events_df.at[i, 'event_number'] = event_numbers
            
    # Initialize new column
    events_df['sound_onset'] = events_df['onset']

    # Find frequent and deviant events
    frequent_mask = events_df['event_name'] == 'frequent'
    deviant_mask = events_df['event_name'] == 'deviant'

    # Get indices of frequent and deviant events
    frequent_indices = events_df[frequent_mask].index
    deviant_indices = events_df[deviant_mask].index

    # Loop over frequent and deviant events
    for index in np.concatenate((frequent_indices, deviant_indices)):
        # Find next sound_in event index
        sound_in_index = events_df.loc[index+1:]['event_name'].eq('sound_in').idxmax()

        # Get onset value of sound_in event
        onset = events_df.loc[sound_in_index, 'onset']

        # Store onset in new_values column
        events_df.loc[index, 'sound_onset'] = onset
        
    # keep only block, frequent and deviant
    events_df = events_df[(events_df['event_name'] == 'deviant') | (events_df['event_name'] == 'frequent') | (events_df['event_name'] == 'block')]
    
    # recode markers number id
    final_event_type_dict = {'block':0, 'frequent':1, 'deviant':2}
    events_df['event_type'] = np.vectorize(final_event_type_dict.get)(events_df['event_name'])
    
    # Create numpy array from filtered dataframe
    final_events = events_df[['sound_onset', 'duration', 'event_type']].to_numpy()
    events_df = events_df.reset_index()
    return final_events, events_df

In [8]:
def crop_useless_data(EEG, events):
    # ONLY KEEP DATA OF INTEREST (WITHIN BLOCKS)
    # Find the indices of events with event_id=0 (beginning of each block)
    event_indices = np.where(events[:, 2] == 0)[0]

    # Define the time boundaries for each block
    t_min_block=[]
    t_max_block=[]

    t_min_block.append((events[0, 0]-3000)/1000) # latency of first event (first block)
    t_max_block.append((events[event_indices[1]-1, 0]+3000)/1000) # latency of last event of first block (last before second block)

    t_min_block.append((events[event_indices[1], 0]-3000)/1000) # latency of second block
    t_max_block.append((events[event_indices[2]-1, 0]+3000)/1000) # latency of last event of second block

    t_min_block.append((events[event_indices[2], 0]-3000)/1000) # latency of third block)
    t_max_block.append((events[-1, 0]+3000)/1000) # latency of last event of third block (last event)

    # Extract the data from each block
    block1 = EEG.copy().crop(t_min_block[0], t_max_block[0])
    block2 = EEG.copy().crop(t_min_block[1], t_max_block[1])
    block3 = EEG.copy().crop(t_min_block[2], t_max_block[2])

    EEG_cropped = mne.concatenate_raws([block1, block2, block3])

#     # update event latencies to match the newly cropped data
#     # Calculate the duration of the removed segments
#     dur_removed_1 = t_min_block[0]*1000 # duration removed before block 1 (ms)
#     dur_removed_2 = round((t_min_block[1] - t_max_block[0])*1000) # duration removed between block 1 and 2 (ms)
#     dur_removed_3 = round((t_min_block[2] - t_max_block[1])*1000) # duration removed between block 2 and 3 (ms)

#     events_cropped = events.copy()
#     # Loop through the events and update their onset values
#     for num, ev in enumerate(events):
#         if ev[0] < t_max_block[0]*1000: # This event occurred during first block
#             events_cropped[num][0] -= dur_removed_1
            
#         elif t_min_block[1]*1000 <= ev[0] < t_max_block[1]*1000: # This event occurred during second block
#             events_cropped[num][0] -= (dur_removed_1+dur_removed_2)
            
#         elif t_min_block[2]*1000 <= ev[0] < t_max_block[2]*1000: # This event occurred during third block
#             events_cropped[num][0] -= (dur_removed_1+dur_removed_2+dur_removed_3)
    
    return EEG_cropped#, events_cropped

In [9]:
def add_events_in_EEG(EEG, events):
    if 'events' in EEG.ch_names:
        EEG.drop_channels(['events'])

    # Create a fake signal to add as a new channel
    new_ch_data = np.zeros((1, len(EEG.times)))
    
    # Create an info object for the new channel
    new_ch_name = ['events']
    new_ch_type = ['stim']
    new_ch_info = mne.create_info(ch_names=new_ch_name, sfreq=EEG.info['sfreq'], ch_types=new_ch_type)
    
    # Create a RawArray object for the new channel data
    new_ch_raw = RawArray(new_ch_data, new_ch_info)
    
    # Add the new channel to the original info object
    EEG.add_channels([new_ch_raw], force_update_info=True)
    EEG.add_events(events, stim_channel='events');
    return EEG

In [10]:
def add_trial_number_in_annotations(raw, events):
    # select only the events that have a non-zero third column
    events = events[events[:, 2] != 0]

    # Create list of annotations with event descriptions and event numbers
    event_desc = ['deviant', 'frequent']  # example event descriptions
    event_numbers = np.arange(1, len(events)+1)  # create a list of event numbers
    annotations_list = []
    for idx, event in enumerate(events):
        desc = event[2]#event_desc[event[2]-1]
        number = event_numbers[idx]-1
        onset = event[0] / raw.info['sfreq']
        duration = 0  # example duration
        annot = (onset, duration, desc, number)
        annotations_list.append(annot)

    # Replace annotations in raw object
    annot = Annotations(onset=[annot[0] for annot in annotations_list],
                        duration=[annot[1] for annot in annotations_list],
                        description=[annot[2] for annot in annotations_list],
                        orig_time=None,
    #                     orig_time=[annot[3] for annot in annotations_list]
                       )
    raw.set_annotations(annot)
    return raw

In [11]:
def keep_freq_before_dev(struct, events_df):
    # Keep epochs of deviants and frequents right before deviants
    if isinstance(struct, mne.io.BaseRaw) or isinstance(struct, mne.Epochs):
        events = mne.find_events(struct)
        # Find all the events identified by "2"
        idx_2 = events[:, 2] == 2
        # Find all the events identified by "1" that are immediately before an event "2"
        idx_1 = np.logical_and(events[:, 2] == 1, np.roll(idx_2, -1))
        # Combine the indices for events identified by "1" and "2"
        idx = np.logical_or(idx_1, idx_2)
        # Select the events that match the indices
        selected_events = events[idx, :]
        final_struct = add_events_in_EEG(struct, selected_events)

    elif isinstance(struct, mne.events.EventMixin):
        events = struct
        # Find all the events identified by "2"
        idx_2 = events[:, 2] == 2
        # Find all the events identified by "1" that are immediately before an event "2"
        idx_1 = np.logical_and(events[:, 2] == 1, np.roll(idx_2, -1))
        # Combine the indices for events identified by "1" and "2"
        idx = np.logical_or(idx_1, idx_2)
        # Select the events that match the indices
        final_struct = events[idx, :]
        
    else:
        print('Unknown data type')
    
    events_df = events_df[(events_df.event_name!='block')]
    events_df = events_df[idx].reset_index()
    return final_struct, events_df

In [12]:
def epochs_to_raw(epochs):
    # Extract the data and metadata from the epochs object
    data = epochs.get_data()

    # Reshape the data to have two dimensions
    n_epochs, n_channels, n_samples = data.shape
    print(n_epochs, n_channels, n_samples)
    data_continuous = np.reshape(data, (n_channels, n_samples * n_epochs))

    # # Create a Raw object from the continuous data
    info = epochs.info
    raw = mne.io.RawArray(data_continuous, info)
    return raw

## Parameters

In [13]:
# Processing parameters
flag_downsample = 1 # 0 for no, 1 for yes
down_freq = 100 # Hertz

bandpass_hard = [1, 30] # in Hz
bandpass_soft = [0.1, 30] # in Hz

epoch_limits = [-0.1, 0.6] # in s
baseline_range = [-0.1, 0] # in s

In [14]:
# Markers information
frequent_marker = 1012 # Response/R 12
deviant_marker = 1013 # Response/R 13
original_marker_list = [frequent_marker, deviant_marker]

new_frequent_marker = 'frequent'
new_deviant_marker = 'deviant'

final_marker_list = [new_frequent_marker, new_deviant_marker]

flag_soundin = 1 # 0 for markers, 1 for sound-in

In [15]:
# Channels management
channels = pd.DataFrame()
channels['old'] = ['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']
channels['new'] = ['Fp1','Fz','F3','F7','FT9','FC5','FC1','C3','T7','TP9','CP5','CP1','Pz','P3','P7','O1','Oz','O2','P4','P8','TP10','CP6','CP2','C4','T8','FT10','FC6','FC2','F4','F8','Fp2','AF7','AF3','AFz','F1','F5','FT7','FC3','C1','C5','TP7','CP3','P1','P5','PO7','PO3','POz','PO4','PO8','P6','P2','CPz','CP4','TP8','C6','C2','FC4','FT8','F6','AF8','AF4','F2','FCz']
new_channels_names = dict(zip(channels['old'], channels['new']))

## Import raw data and format it to preprocessing pipeline

In [16]:
eeg_file = 'bob'


In [17]:
# Parameters
eeg_file = "D:/RAW DATA/2021-own-name-revcor/EEG\\Revcor0020.vhdr"


In [18]:
# eeg_file = 'D:/RAW DATA/2021-own-name-revcor/EEG/Revcor0011.vhdr'
print(eeg_file)

subject_number = eeg_file.split('/')[-1].split('.')[0][-4:]
print(subject_number)

# Load data
EEG = mne.io.read_raw_brainvision(eeg_file, 
                               scale=1.0, 
                               preload=True, 
                               verbose=None)

# Rename the channels
EEG = EEG.rename_channels(new_channels_names)

# Load the electrode locations for 64 channels
montage = mne.channels.make_standard_montage('easycap-M1')

# Apply the electrode locations to the data
EEG = EEG.set_montage(montage)
# fig = EEG.plot_sensors(show_names=True)

# load events
events, _ = mne.events_from_annotations(EEG, event_id='auto')

# Check events if necessary
counts_markers = check_markers(events, original_marker_list)

# replace markers id so it is clearer, and taking the "sound-in" marker immediately after identification marker 
events, events_df = take_sound_in(events)
# events = events[np.argsort(events[:, 0])]

# insert events into raw structure
EEG = add_events_in_EEG(EEG, events)

# # add trial number in annotations to ensure correspondance with behavioural data
# EEG = add_trial_number_in_annotations(EEG, events)
    
# Only keep data of interest (within blocks)
EEG = crop_useless_data(EEG, events)

# Check for NaN values in the raw data
EEG, sum_nan_2 = checkNan(EEG)

# Keep only frequents before deviants to save memory
EEG, events_df = keep_freq_before_dev(EEG, events_df)

# Copy EEG for hard processing
EEG_hard = EEG.copy()

D:/RAW DATA/2021-own-name-revcor/EEG\Revcor0020.vhdr
0020
Extracting parameters from D:/RAW DATA/2021-own-name-revcor/EEG\Revcor0020.vhdr...


Setting channel info structure...


Reading 0 ... 3273739  =      0.000 ...  3273.739 secs...


Used Annotations descriptions: ['New Segment/', 'Response/R  1', 'Response/R  2', 'Response/R  3', 'Response/R  4', 'Response/R  5', 'Response/R  6', 'Response/R  7', 'Response/R  8', 'Response/R  9', 'Response/R 11', 'Response/R 12', 'Response/R 13', 'Response/R 15', 'Stimulus/S  2']


Creating RawArray with float64 data, n_channels=1, n_times=3273740


...

3000 events found


Event IDs: [1 2]


Creating RawArray with float64 data, n_channels=1, n_times=3021599


    Range : 0 ... 3021598 =      0.000 ...  3021.598 secs


Ready.


## Preprocessing pipeline

### Part 1: soft processing

#### Filtering

In [19]:
# Define the Butterworth filter parameters
iir_params = dict(order=2, ftype='butter', output='sos')
iir_params = mne.filter.construct_iir_filter(iir_params,
                                             f_pass=bandpass_soft, 
                                             f_stop=None, 
                                             sfreq=EEG.info['sfreq'], 
                                             btype='bandpass', 
                                             return_copy=False)
EEG.filter(bandpass_soft[0], bandpass_soft[1], method='iir', iir_params=iir_params);

IIR filter parameters


---------------------


Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:


- Filter order 8 (effective, after forward-backward)


- Cutoffs at 0.10, 30.00 Hz: -6.02, -6.02 dB


Filtering raw data in 3 contiguous segments


Setting up band-pass filter from 0.1 - 30 Hz


#### Notch filter

In [20]:
# Apply notch filter
freqs = np.arange(50, EEG.info['sfreq']/2, 50)
EEG.notch_filter(freqs=freqs, fir_design='firwin');

Setting up band-stop filter


FIR filter parameters


---------------------


Designing a one-pass, zero-phase, non-causal bandstop filter:


- Windowed time-domain design (firwin) method


...

- Upper transition bandwidth: 0.50 Hz


- Filter length: 6601 samples (6.601 sec)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.2s remaining:    0.0s


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    3.8s finished


#### Divide data into epochs

In [21]:
# divide data into real epochs
event_id = {'frequent':1,
           'deviant':2}
events = mne.find_events(EEG, stim_channel='events', min_duration=0.001)
epochs = mne.Epochs(EEG, 
                    events, 
                    event_id=event_id,
                    tmin=epoch_limits[0], 
                    tmax=epoch_limits[1], 
                    reject=None,
                    preload=True, 
                    baseline=None)
# epochs.load_data(reject=None)


1500 events found


Event IDs: [1 2]


Not setting metadata


1500 matching events found


No baseline correction applied


0 projection items activated


Using data from preloaded Raw for 1500 events and 701 original time points ...


0 bad epochs dropped


#### Reject bad channels

https://autoreject.github.io/stable/auto_examples/plot_ransac.html#sphx-glr-auto-examples-plot-ransac-py

In [22]:
ransac = Ransac(verbose=True, 
                picks=channels.new.to_list(), 
                n_jobs=16)
epochs = ransac.fit_transform(epochs)
epochs.info['bads'] = ransac.bad_chs_
# epochs.drop_channels(ransac.bad_chs_);

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.


[Parallel(n_jobs=16)]: Done   3 out of  16 | elapsed:    2.1s remaining:    9.6s
[Parallel(n_jobs=16)]: Done   5 out of  16 | elapsed:    2.2s remaining:    4.9s
[Parallel(n_jobs=16)]: Done   7 out of  16 | elapsed:    2.2s remaining:    2.8s
[Parallel(n_jobs=16)]: Done   9 out of  16 | elapsed:    2.2s remaining:    1.7s
[Parallel(n_jobs=16)]: Done  11 out of  16 | elapsed:    2.2s remaining:    0.9s
[Parallel(n_jobs=16)]: Done  13 out of  16 | elapsed:    2.2s remaining:    0.4s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    2.2s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.


[Parallel(n_jobs=16)]: Done   3 out of  16 | elapsed:   11.8s remaining:   51.6s
[Parallel(n_jobs=16)]: Done   5 out of  16 | elapsed:   11.9s remaining:   26.2s
[Parallel(n_jobs=16)]: Done   7 out of  16 | elapsed:   11.9s remaining:   15.4s
[Parallel(n_jobs=16)]: Done   9 out of  16 | elapsed:   11.9s remaining:    9.3s
[Parallel(n_jobs=16)]: Done  11 out of  16 | elapsed:   11.9s remaining:    5.4s
[Parallel(n_jobs=16)]: Done  13 out of  16 | elapsed:   12.0s remaining:    2.7s


[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:   12.0s finished


[Done]
Interpolating bad channels


    Automatic origin fit: head of radius 95.0 mm


Computing interpolation matrix from 57 sensor positions


Interpolating 6 sensors


#### Clean artifacts

In [23]:
# Initialize the AutoReject algorithm
n_interpolates = [1, 2, 32, 60]
# consensus_percs = 0.6
ar = AutoReject(n_interpolates,
                consensus=[0.6],
                thresh_method='random_search',
                random_state=11, 
                n_jobs=1, 
                verbose=True)

ar.fit(epochs[:100])  # fit on a few epochs to save time
epochs, reject_log = ar.transform(epochs, return_log=True);

C:\Program Files\Python310\lib\site-packages\autoreject\utils.py:65: UserWarning: 6 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  warnings.warn(


Running autoreject on ch_type=eeg


  0%|          | Creating augmented epochs : 0/57 [00:00<?,       ?it/s]

  0%|          | Computing thresholds ... : 0/57 [00:00<?,       ?it/s]

  0%|          | Repairing epochs : 0/100 [00:00<?,       ?it/s]

...

  0%|          | Fold : 0/10 [00:00<?,       ?it/s]





Estimated consensus=0.60 and n_interpolate=32


C:\Program Files\Python310\lib\site-packages\autoreject\utils.py:65: UserWarning: 6 channels are marked as bad. These will be ignored. If you want them to be considered by autoreject please remove them from epochs.info["bads"].
  warnings.warn(


  0%|          | Repairing epochs : 0/1500 [00:00<?,       ?it/s]

Dropped 72 epochs: 46, 161, 205, 286, 287, 290, 291, 363, 370, 382, 404, 426, 458, 475, 481, 565, 576, 651, 658, 677, 697, 700, 714, 729, 732, 766, 829, 848, 850, 852, 870, 889, 892, 899, 911, 918, 935, 938, 954, 977, 988, 990, 995, 1101, 1105, 1107, 1110, 1112, 1122, 1148, 1156, 1161, 1173, 1179, 1184, 1189, 1211, 1240, 1246, 1247, 1258, 1268, 1282, 1317, 1342, 1358, 1383, 1445, 1467, 1488, 1489, 1496


In [24]:
search_str = 'AUTOREJECT',
indices = []

for i, elem in enumerate(epochs.drop_log):
    if elem == search_str:
        indices.append(i)

# print(indices)

events_df.drop(index=indices)

,level_0,index,onset,duration,event_type,event_name,event_number,sound_onset
0,11,62,23405,0,1,frequent,0010,23600
1,12,68,24303,0,2,deviant,0011,24608
2,21,122,32857,0,1,frequent,0020,33051
3,22,128,33738,0,2,deviant,0021,34023
4,24,140,35984,0,1,frequent,0023,36178
...,...,...,...,...,...,...,...,...
1494,2980,18042,3241609,0,1,frequent,2977,3241802
1495,2981,18048,3242460,0,2,deviant,2978,3242752
1497,2985,18072,3246565,0,2,deviant,2982,3246864
1498,2990,18102,3251405,0,1,frequent,2987,3251598


### Part 2: hard processing for ICA

#### Downsample for ICA

In [25]:
if flag_downsample == 1:
    EEG_hard.resample(down_freq, npad='auto'); # set sampling frequency to X points per second

1500 events found


Event IDs: [1 2]


1500 events found


Event IDs: [1 2]


<RawBrainVision | Revcor0020.eeg, 64 x 302160 (3021.6 s), ~147.6 MB, data loaded>

#### Filtering

In [26]:
# Define the Butterworth filter parameters
iir_params = dict(order=2, ftype='butter', output='sos')
iir_params = mne.filter.construct_iir_filter(iir_params,
                                             f_pass=bandpass_hard, 
                                             f_stop=None, 
                                             sfreq=EEG_hard.info['sfreq'], 
                                             btype='bandpass', 
                                             return_copy=False)
EEG_hard.filter(bandpass_hard[0], bandpass_hard[1], method='iir', iir_params=iir_params);

IIR filter parameters


---------------------


Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:


- Filter order 8 (effective, after forward-backward)


- Cutoffs at 1.00, 30.00 Hz: -6.02, -6.02 dB


Filtering raw data in 3 contiguous segments


Setting up band-pass filter from 1 - 30 Hz


#### Notch filter

In [27]:
# Apply notch filter
freqs = 49
EEG_hard.notch_filter(freqs=freqs, fir_design='firwin');

Setting up band-stop filter from 48 - 50 Hz


FIR filter parameters


---------------------


Designing a one-pass, zero-phase, non-causal bandstop filter:


- Windowed time-domain design (firwin) method


...

- Upper passband edge: 49.62 Hz


- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.87 Hz)


- Filter length: 661 samples (6.610 sec)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.3s finished


#### Divide data into epochs

In [28]:
# divide data into real epochs
event_id = {'frequent':1,
           'deviant':2}
events = mne.find_events(EEG_hard, stim_channel='events', min_duration=0.001)
epochs_hard = mne.Epochs(EEG_hard, 
                    events, 
                    event_id=event_id,
                    tmin=epoch_limits[0], 
                    tmax=epoch_limits[1], 
                    reject=None,
                    preload=True, 
                    baseline=None)

1500 events found


Event IDs: [1 2]


Not setting metadata


1500 matching events found


No baseline correction applied


0 projection items activated


Using data from preloaded Raw for 1500 events and 71 original time points ...


0 bad epochs dropped


#### Reject bad channels (selected in part 1)

In [29]:
# Mark channels as bad and drop them (full rank ICA)
epochs_hard.info['bads'] = ransac.bad_chs_
epochs_hard.drop_channels(ransac.bad_chs_);

#### Clean artifacts

In [30]:
# Initialize the AutoReject algorithm
# n_interpolates = [1, 2, 32, 60]
# # consensus_percs = 0.6
# ar = AutoReject(n_interpolates,
#                 consensus=[0.6],
#                 thresh_method='random_search',
#                 random_state=11, 
#                 n_jobs=1, 
#                 verbose=True)

ar.fit(epochs_hard[:100])  # fit on a few epochs to save time
epochs_hard, reject_log = ar.transform(epochs_hard, return_log=True);

Running autoreject on ch_type=eeg


  0%|          | Creating augmented epochs : 0/57 [00:00<?,       ?it/s]

  0%|          | Computing thresholds ... : 0/57 [00:00<?,       ?it/s]

  0%|          | Repairing epochs : 0/100 [00:00<?,       ?it/s]

  0%|          | n_interp : 0/4 [00:00<?,       ?it/s]

...

  0%|          | Repairing epochs : 0/100 [00:00<?,       ?it/s]

  0%|          | Fold : 0/10 [00:00<?,       ?it/s]





Estimated consensus=0.60 and n_interpolate=32


  0%|          | Repairing epochs : 0/1500 [00:00<?,       ?it/s]

Dropped 102 epochs: 46, 67, 161, 165, 205, 286, 287, 290, 291, 301, 305, 307, 364, 370, 382, 398, 403, 404, 426, 444, 458, 481, 565, 576, 651, 658, 659, 677, 697, 705, 714, 729, 732, 753, 766, 818, 819, 829, 830, 838, 839, 848, 850, 869, 889, 892, 897, 898, 911, 918, 935, 938, 953, 954, 975, 977, 988, 990, 1043, 1046, 1064, 1076, 1077, 1101, 1103, 1105, 1107, 1110, 1112, 1117, 1122, 1141, 1148, 1149, 1156, 1161, 1166, 1173, 1175, 1179, 1184, 1189, 1211, 1240, 1241, 1246, 1247, 1258, 1268, 1282, 1285, 1317, 1342, 1380, 1383, 1400, 1445, 1460, 1467, 1488, 1489, 1496


#### ICA

https://pierreablin.github.io/picard/auto_examples/plot_ecg_detection.html

In [31]:
# Set up and fit the ICA
ica = ICA(method='picard', fit_params=dict(ortho=False, extended=True))
ica.fit(epochs_hard, decim=4);

ic_labels = label_components(epochs_hard, ica, method="iclabel")
labels = ic_labels["labels"]

exclude_idx = [idx for idx, label in enumerate(labels) if label not in ["brain", "other"]]
print(f"Excluding these ICA components: {exclude_idx}")

# ica.plot_properties(epochs_hard, picks=[exclude_idx], verbose=False)

Fitting ICA to data using 57 channels (please be patient, this may take a while)


Selecting by non-zero PCA components: 57 components


Fitting ICA took 8.0s.


Method,picard
Fit,86 iterations on epochs (25164 samples)
ICA components,57
Available PCA components,57
Channel types,eeg
ICA components marked for exclusion,—


G:\Utilisateurs\c.joucla\AppData\Local\Temp\ipykernel_3056\2871084106.py:5: RuntimeWarning: The provided Epochs instance does not seem to be referenced to a common average reference (CAR). ICLabel was designed to classify features extracted from an EEG dataset referenced to a CAR (see the 'set_eeg_reference()' method for Raw and Epochs instances).
  ic_labels = label_components(epochs_hard, ica, method="iclabel")
G:\Utilisateurs\c.joucla\AppData\Local\Temp\ipykernel_3056\2871084106.py:5: RuntimeWarning: The provided Epochs instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  ic_labels = label_components(epochs_hard, ica, method="iclabel")


Excluding these ICA components: [7, 46, 50]


In [32]:
# # Initialize the AutoReject algorithm
# n_interpolates = [1, 2, 32, 60]
# # consensus_percs = 0.6
# ar = AutoReject(n_interpolates,
#                 consensus=[0.6],
#                 thresh_method='random_search',
#                 random_state=11, 
#                 n_jobs=1, 
#                 verbose=True)

# ar.fit(epochs[:100])  # fit on a few epochs to save time
# epochs, reject_log = ar.transform(epochs, return_log=True);

### Part 3: Transfer ICA weights to soft EEG

#### Remove ICA components from soft EEG

In [33]:
ica.apply(epochs, exclude=exclude_idx);

Applying ICA to Epochs instance


    Transforming to ICA space (57 components)


    Zeroing out 3 ICA components


    Projecting back using 57 PCA components


#### Channels interpolation

In [34]:
# Interpolate bad channels
epochs.info['bads'] = ransac.bad_chs_
epochs.interpolate_bads(reset_bads=True);

Interpolating bad channels


    Automatic origin fit: head of radius 95.0 mm


Computing interpolation matrix from 57 sensor positions


Interpolating 6 sensors


#### Add Cz

In [35]:
epochs.add_reference_channels('Cz');

G:\Utilisateurs\c.joucla\AppData\Local\Temp\ipykernel_3056\2603832755.py:1: RuntimeWarning: The locations of multiple reference channels are ignored.
  epochs.add_reference_channels('Cz');


#### Avg ref

In [36]:
epochs.set_eeg_reference('average', projection=True)
epochs.apply_proj();

EEG channel type selected for re-referencing


Adding average EEG reference projection.


1 projection items deactivated


Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.


Created an SSP operator (subspace dimension = 1)


1 projection items activated


SSP projectors applied...


#### Add Cz coordinates

In [37]:
# reset chans coordinates
epochs = epochs.set_montage(montage)

#### Removing baseline

In [38]:
epochs.apply_baseline(baseline=(-0.1, 0), verbose=False);

### Save file

In [39]:
path_save = './data/own_name_revcor_preprocessedEEG_subj_'+subject_number+'.fif'
epochs.save(path_save, overwrite=True)

events_df.to_csv('./data/own_name_revcor_preprocessedEEG_events_subj_'+subject_number+'.csv')

Overwriting existing file.


G:\Utilisateurs\c.joucla\AppData\Local\Temp\ipykernel_3056\1323138735.py:2: RuntimeWarning: This filename (./data/own_name_revcor_preprocessedEEG_subj_0020.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(path_save, overwrite=True)
